In [2]:
import torch
import torchaudio 
import os
import librosa
import matplotlib.pyplot as plt
from torchaudio import transforms
from IPython.display import Audio

In [1]:
from classifier.audioutils import AudioUtils

In [10]:
# Load audio input data into map

# NOTE: Assumes all samples are of the sample sample rate - if this is not the case add a method to AudioUtils to convert to 44100 (or whatever!)

CURRENT_DIR = os.getcwd()
AUDIO_DIR = os.path.join(CURRENT_DIR, "audio_input")

# Scan AUDIO_DIR and load all filenames into a map with key - path, value - label (1-10, 1. kick, 2. snare, ... 10.cymbal)

file_map = {}
for subdir, dirs, files in os.walk(AUDIO_DIR):
    for file in files:
        if os.path.splitext(file)[-1] == ".wav":
            # get label for subdir
            label = int(os.path.split(subdir)[-1].split("_")[0])
            path = os.path.join(subdir, file)
            file_map[path] = label

print("Number of input files: ", len(file_map))

Number of input files:  2244


In [11]:
# Calculate the average length and use this to pad or truncate audio files to all be the same size
max_length = 0
lengths = []
for key in file_map:
    sig, sr = AudioUtils.open(key)
    max_length = max(max_length, sig.shape[1])
    lengths.append(sig.shape[1])

average_length = sum(lengths) / len(file_map)
desired_sample_len = int(average_length)

print("Max Length:", max_length, "Average Length:", average_length, "Sample Size: ", desired_sample_len)


Max Length: 341501 Average Length: 20734.03787878788 Sample Size:  20734


In [12]:
# This is basically just testing each of the transformation methods
for key in file_map:
    sig, sr = AudioUtils.open(key)
    sig = AudioUtils.convert_mono(sig)
    sig = AudioUtils.resize(sig, desired_sample_len)